# Building Kernels and Applying Gates

This section will cover the most basic CUDA-Q construct, a quantum kernel. Topics include, building kernels, initializing states, and applying gate operations.

### Defining Kernels

Kernels are the building blocks of quantum algorithms in CUDA-Q. A kernel is specified by using the following syntax. `cudaq.qubit` builds a register consisting of a single qubit, while `cudaq.qvector` builds a register of $N$ qubits.

In [1]:
import cudaq

In [2]:
@cudaq.kernel
def kernel():
    A = cudaq.qubit()
    B = cudaq.qvector(3)
    C = cudaq.qvector(5)

Inputs to kernels are defined by specifying a parameter in the kernel definition along with the appropriate type. The kernel below takes an integer to define a register of N qubits.

In [2]:
N = 2

@cudaq.kernel
def kernel(N: int):
    register = cudaq.qvector(N)

NameError: name 'cudaq' is not defined

### Initializing states

It is often helpful to define an initial state for a kernel. There are a few ways to do this in CUDA-Q. Note, method 5 is particularly useful for cases where the state of one kernel is passed into a second kernel to prepare its initial state.

1. Passing complex vectors as parameters
2. Capturing complex vectors
3. Precision-agnostic API
4. Define as CUDA-Q amplitudes
5. Pass in a state from another kernel

In [2]:
c = [.707, -.707j]

@cudaq.kernel
def kernel():
    q = cudaq.qvector(c)


print(cudaq.get_state(kernel))

TypeError: get(): incompatible function arguments. The following argument types are supported:
    1. (type: cudaq.mlir._mlir_libs._mlir.ir.Type, value: float, loc: mlir.ir.Location = None) -> cudaq.mlir._mlir_libs._mlir.ir.FloatAttr

Invoked with: Type(f64), (-0-0.707j)

In [4]:
# Passing complex vectors as parameters
c = [.707 +0j, 0-.707j]

@cudaq.kernel
def kernel(vec: list[complex]):
    q = cudaq.qubit(vec)


# Capturing complex vectors
c = [0.70710678 + 0j, 0., 0., 0.70710678]

@cudaq.kernel
def kernel():
    q = cudaq.qvector(c)


# Precision-Agnostic API
import numpy as np

c = np.array([0.70710678 + 0j, 0., 0., 0.70710678], dtype=cudaq.complex())

@cudaq.kernel
def kernel():
    q = cudaq.qvector(c)

# Define as CUDA-Q amplitudes
c = cudaq.amplitudes([0.70710678 + 0j, 0., 0., 0.70710678])

@cudaq.kernel
def kernel():
    q = cudaq.qvector(c)

# Pass in a state from another kernel
c = [0.70710678 + 0j, 0., 0., 0.70710678]

@cudaq.kernel
def kernel_initial():
    q = cudaq.qvector(c)

state_to_pass = cudaq.get_state(kernel_initial)

@cudaq.kernel
def kernel(state: cudaq.State):
    q = cudaq.qvector(state)

kernel(state_to_pass)

### Applying Gates


After a kernel is constructed, gates can be applied to start building out a quantum circuit. The image below shows all of the predefined CUDA-Q gates which are explained in detail in the API [here](https://nvidia.github.io/cuda-quantum/latest/api/default_ops.html#unitary-operations-on-qubits).


![Htest](../images/gates.png)


Gates can be applied to all qubits in a register:

In [8]:
@cudaq.kernel
def kernel():
    register = cudaq.qvector(10)
    h(register)

Or, to individual qubits in a register:

In [9]:
@cudaq.kernel
def kernel():
    register = cudaq.qvector(10)
    h(register[0])  # first qubit
    h(register[-1])  # last qubit

### Controlled Operations

Controlled operations are available for any gate and can be used by adding `.ctrl` to the end of any gate, followed by specification of the control qubit and the target qubit.

In [10]:
@cudaq.kernel
def kernel():
    register = cudaq.qvector(10)
    x.ctrl(register[0], register[1])  # CNOT gate applied with qubit 0 as control

### Multi-Controlled Operations

It is valid for more than one qubit to be used for multi-controlled gates. The control qubits are specified as a list.

In [11]:
@cudaq.kernel
def kernel():
    register = cudaq.qvector(10)
    x.ctrl([register[0], register[1]], register[2])  # X applied to qubit two controlled by qubit 0 and 1

### Adjoit Operations

The adjoint of a gate can be applied by appending the gate with the `adj` designation.

In [12]:
@cudaq.kernel
def kernel():
    register = cudaq.qvector(10)
    t.adj(register[0])

### Custom Operations

Custom gate operations can be specified using `cudaq.register_operation`. A one-dimensional Numpy array specifies the unitary matrix to be applied. The entries of the array read from top to bottom through the rows.

In [13]:
import numpy as np

cudaq.register_operation("custom_x", np.array([0, 1, 1, 0]))

@cudaq.kernel
def kernel():
    qubits = cudaq.qvector(2)
    h(qubits[0])
    custom_x(qubits[0])
    custom_x.ctrl(qubits[0], qubits[1])

### Measurement

Kernel measurement can be specified in the Z, X, or Y basis using `mz`, `mx`, and `my`. If a measurement is specified with no argument, the entire kernel is measured in that basis. Measurement occurs in the Z basis by default.

In [14]:
@cudaq.kernel
def kernel():
    qubits = cudaq.qvector(2)
    mz()

Specific qubits or registers can be measured rather than the entire kernel.

In [15]:
@cudaq.kernel
def kernel():
    qubits_a = cudaq.qvector(2)
    qubit_b = cudaq.qubit()
    mz(qubits_a)
    mx(qubit_b)

### Midcircuit Measurement and Conditional Logic

In certain cases, it it is helpful for some operations in a quantum kernel to depend on measurement results following previous operations. This is accomplished in the following example by performing a Hadamard on qubit 0, then measuring qubit 0 and savig the result as `b0`. Then, an if statement performs a Hadamard on qubit 1 only if `b0` is 1. Measuring this qubit 1 verifies this process as a 1 is the result 25% of the time.

In [ ]:
@cudaq.kernel
def kernel():
    q = cudaq.qvector(2)
    h(q[0])
    b0 = mz(q[0])
    if b0:
        h(q[1])
        mz(q[1])

### Building Kernels with Kernels

For many complex applications, it is helpful for a kernel to call another kernel to perform a specific subroutine. The example blow shows how `kernel_A` can be caled within `kernel_B` to perform CNOT operations.

In [16]:
@cudaq.kernel
def kernel_A(qubit_0: cudaq.qubit, qubit_1: cudaq.qubit):
    x.ctrl(qubit_0, qubit_1)

@cudaq.kernel
def kernel_B():
    reg = cudaq.qvector(10)
    for i in range(5):
        kernel_A(reg[i], reg[i + 1])

### Parameterized Kernels

It is often useful to define parameterized circuit kernels which can be used for applications like VQE.

In [17]:
@cudaq.kernel
def kernel(thetas: list[float]):
    qubits = cudaq.qvector(2)
    rx(thetas[0], qubits[0])
    ry(thetas[1], qubits[1])

thetas = [.024, .543]

kernel(thetas)